In [2]:
import pandas as pd
import numpy as np
import os
import openpyxl as xl


import re


import warnings
warnings.filterwarnings('ignore')


from Utilities import FileIO
from Utilities import DataframeCleaning

# Load Initial Files

In [3]:

    
volExtractFileName = "extractFinished_volume.csv"
costExtractFileName = "extractFinished_cost.csv"

df_vol= FileIO.importFile(volExtractFileName)
df_cost = FileIO.importFile(costExtractFileName)
# df_vol.head()
df_cost.head()

,product,revenue,attributable_cost,volume_variable_cost,product_specific_cost,rev_pc,attributable_cost_pc,cont_pc,cost_coverage,fy
0,Mail Classes and Products,Revenue,Cost,Variable Cost,Specific Cost,$,Cost $,$,Coverage,2008
1,(note 1),(note 1),(note 2),(note 2),(note 2),NaN,(note 2),(note 2),(note 2),2008
2,MARKET DOMINANT MAIL,A,B,C,F,D,E,(D-E),(D/E),2008
3,First-Class Mail:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008
4,Single-Piece Letters,14353.524033,8443.075618748211,8434.32261874821,8.753,0.4283392480234627,0.2519590766159719,0.1763801714074908,1.7000349968591284,2008


In [4]:
# df_cost[df_cost['product'].isnull()]
# df_cost[df_cost['product'].str.contains("Ad")][['product','fy']]
# df_cost[df_cost['product'].str.contains("note")][['product','fy']]
df_cost[df_cost['product'].str.contains("Ch")][['product','fy']]


,product,fy
131,Change of Address Credit Card Authen. (note 4),2009
209,Change of Address Credit Card Authentication,2010
294,Change of Address Credit Card Authentication,2011
379,Change of Address Credit Card Authentication,2012


# Clean Product Names 


In [5]:

def removeSubstrings(frame):

    frame['product']  =frame['product'].str.replace("(Note \d+|\(Note\))", "", case=False, regex=True)

        # # remove these chars
    deleteChars = [".", "(New)", ":", ",", "()"]
    pattern = '|'.join(map(re.escape, deleteChars))
    frame['product'] = frame['product'].str.replace(pattern, '', regex=True)


    frame = frame[frame['product'].notnull()]
    frame['product'] = frame['product'].str.strip()
    return frame



df_vol= removeSubstrings(df_vol)
df_cost= removeSubstrings(df_cost)

# df_cost[df_cost['product'].str.contains("Ad")]
# df_cost[df_cost['product'].str.contains("note")]

In [6]:

# df_vol = df_vol_removeChars.copy()
# df_cost = df_cost_removeChars.copy()


def removeChars_doubleCheck(frame):
    filterFrame = frame[frame['product'].str.contains("Note")]
    if len(filterFrame) >0:
        frame = removeSubstrings(frame)
    
    return frame 



def cleanProductNameCol(frame):
    def applyClean(row):
        row = str(row)
        row = row.strip()

        andChars = ["/", "&"]
        for char in andChars:
            row  = row.replace(char, " and ")

        spaceChars = ["-"]
        for char in spaceChars:
            row  = row.replace(char, " ")


        internationalSynonyms = ["Intl", "Int'l"]
        for char in internationalSynonyms: 
            row = row.replace(char, "International")

        row = re.sub(r'\s{2,}', ' ', row)

        row = row.title()

        acronyms=["Nsa", "Nsas", "Prs", "Cra", "Upu", "Lr1", "Cod", "Csc", "Usps", "Us"]
        for acro in acronyms:
            row  = row.replace(acro, acro.upper())
        
        lowerCase = [" And ", " At", " Of "]
        lowerCaseFinalList = []
        for word in lowerCase:
            lowerCaseFinalList.append(word)
            #handle exception for cases like "(and x)" and  "Post Office"
            lowerCaseFinalList.append("("+word[1:])

        for word in lowerCaseFinalList:
            row = row.replace(word, word.lower())

        row = row.strip()
        return row

    frame['product'] =frame['product'].apply(lambda row: applyClean(row))

    removeChars_doubleCheck(frame)
    frame['product'] = frame['product'].str.strip()
    return frame


df_cost_clean = cleanProductNameCol(df_cost)
df_vol_clean = cleanProductNameCol(df_vol)



# Handle Fees
for now, label all fees as first class, they will be differentiated in the "duplicates" section

In [7]:
df_cost = df_cost_clean.copy()
df_vol = df_vol_clean.copy()

def labelFees(frame):
   frame.loc[frame['product']=='Fees', 'product'] = "First Class Fees"
   return frame 


df_cost_fees = labelFees(df_cost)
df_vol_fees = labelFees(df_vol)

In [8]:
# df_vol_fees[df_vol_fees['product'].str.contains("Service Transactions")]

# Match Product Names

In [9]:
keyFrame = FileIO.importKey()

In [10]:
df_cost = df_cost_fees.copy()
df_vol= df_vol_fees.copy()

def matchProductName(frame):

    allFinalRowDicts = []

    keyMatchList = list(keyFrame['product'])  
    

    for rowIdx in range(len(frame)):
        rowDict = frame.iloc[rowIdx].to_dict()
        productToMatch= rowDict['product']

        isMatchFound = False


        if productToMatch in keyMatchList:
            isMatchFound = True
            
            matchingKeyIndex = keyFrame[keyFrame['product']==productToMatch].index[0]
            keyRowDict = keyFrame.loc[matchingKeyIndex].to_dict()

            matchingProductId  = keyRowDict['product_id']
            keyProductMatch= keyRowDict['product']
        
        if not isMatchFound:
            for keyIdx, keyRow in keyFrame.iterrows():
                aliasCellString = str(keyRow['alias_list'])

                aliasCellList = aliasCellString.split(", ")

                if productToMatch in aliasCellList:
                    isMatchFound = True
                    matchingProductId = keyRow['product_id']
                    keyProductMatch = keyRow['product']

                    break
        if not isMatchFound:
            matchingProductId, keyProductMatch = 0, 'not found'

        # does not match LR1 definition of total special services 
        if productToMatch=="Total Special Services":
            matchingProductId = 0

        rowDictFinal = rowDict
        rowDictFinal['product_id'] = matchingProductId
        # rowDictFinal['keyProductMatch'] =keyProductMatch
        
        allFinalRowDicts.append(rowDictFinal)


    dfs = [pd.DataFrame(data_dict, index=[0]) for data_dict in allFinalRowDicts]

    # Concatenate the DataFrames into a single DataFrame
    rez = pd.concat(dfs, ignore_index=True)

    return rez

    

df_cost_matched = matchProductName(df_cost)
df_vol_matched = matchProductName(df_vol)


# Drop Blank Rows

In [11]:
df_cost = df_cost_matched.copy()
df_vol = df_vol_matched.copy()
allContinuousCols = ['volume', 'weight', 'weight_pc','revenue', 'attributable_cost', 'volume_variable_cost',
       'product_specific_cost', 'rev_pc', 'attributable_cost_pc', 'cont_pc',
       'cost_coverage']


def dropBlankRows(frame):


    allFrameCols= frame.columns

    frameCols_cont = [c for c in frame.columns if c in allContinuousCols]

    frame = frame.dropna(subset = frameCols_cont, how='all')

    return frame


df_cost_dropNull =  dropBlankRows(df_cost)
df_vol_dropNull =  dropBlankRows(df_vol)

In [12]:
#  unmatchedSer = df_cost_dropNull[df_cost_dropNull['product_id']==0]['product'].value_counts()
# # unmatchedSer = df_vol_dropNull[df_vol_dropNull['product_id']==0]['product'].value_counts()
# #  df_vol_dropNull[df_vol_dropNull['product']=='Total Special Services']
# # # df_cost_dropNull[df_cost_dropNull['product']=='USPS Marketing Mail']

# unmatchedSer.index[:20]

df_cost = df_cost_dropNull.copy()

df_cost[df_cost["product_id"]==0]['product'].value_counts()

product
Total                                                30
Mail Classes and Products                            21
                                                     21
Total Market Dominant Mail                           16
Total Special Services                               16
Total Market Dominant Mail and Services              16
Total All Mail and Services                          16
Appropriations Revenue Forgone                       16
Miscellaneous Items                                  15
All Other                                            14
Total Operating Income                               13
Market Dominant Mail                                  5
Total Competitive International Mail and Services     5
Total Market Dominant Special Services                4
Competitive Mail and Services                         4
Total All                                             2
Total Operating Revenue                               2
Total Competitive Services              

# Handle Total Rows 

In [26]:
df_cost[df_cost["product"].str.contains("Total")]['product'].value_counts()

product
Total Single Piece Letters and Cards                        16
Total Presort Letters and Cards                             16
Total First Class (incl International)                      16
Total Marketing Mail (incl International)                   16
Total Periodicals (incl International)                      16
Total Package Services (incl International)                 16
Total International Ancillary Services                      16
Total Competitive Mail and Services (Incl International)    16
Total Domestic Ancillary Services                           15
Total Special Services (incl International)                 15
Total Ground                                                15
Total Domestic Competitive Services                         14
Name: count, dtype: int64

# Drop Unmatched

In [14]:
df_cost = df_cost_dropNull.copy()
df_vol = df_vol_dropNull.copy()

def dropUnmatchedProducts(frame):
    return frame[frame['product_id']!=0]


df_cost_dropUnmatched = dropUnmatchedProducts(df_cost)
df_vol_dropUnmatched = dropUnmatchedProducts(df_vol)


In [15]:
df_cost_dropNull[df_cost_dropNull['product_id']==0]['product'].value_counts()

product
Total                                                30
Mail Classes and Products                            21
                                                     21
Total Market Dominant Mail                           16
Total Special Services                               16
Total Market Dominant Mail and Services              16
Total All Mail and Services                          16
Appropriations Revenue Forgone                       16
Miscellaneous Items                                  15
All Other                                            14
Total Operating Income                               13
Market Dominant Mail                                  5
Total Competitive International Mail and Services     5
Total Market Dominant Special Services                4
Competitive Mail and Services                         4
Total All                                             2
Total Operating Revenue                               2
Total Competitive Services              

# Align with Key

In [16]:
df_cost = df_cost_dropUnmatched.copy()
df_vol = df_vol_dropUnmatched.copy()



df_cost_realigned = DataframeCleaning.realignWithKey(df_cost, keyFrame)
df_vol_realigned = DataframeCleaning.realignWithKey(df_vol, keyFrame)



# Duplicate Product Names

In [17]:
df_cost = df_cost_realigned.copy()
df_vol = df_vol_realigned.copy()

# costSer = df_cost.groupby(["product_id", "fy"])['product'].value_counts()
# costFrame = pd.DataFrame(costSer)
# costFrame = costFrame.reset_index()
# costFrame = costFrame[costFrame['count']>1]
# costFrame['product'].value_counts()

In [18]:
allDuplicateTypes  = keyFrame['duplicate_name'].value_counts().index
allDuplicateProducts = list(keyFrame[keyFrame['duplicate_name'].notnull()]['product'])
allDuplicateDictionaries = []

for duplicate in allDuplicateTypes:
    dupFrame = keyFrame[keyFrame['duplicate_name']==duplicate]
    dupIds = list(dupFrame['product_id'])
    dupMailClasses = list(dupFrame['mail_class'])
    dupDictionary = dict(zip(dupIds, dupMailClasses))
    allDuplicateDictionaries.append(dupDictionary)

# allDuplicateDictionaries


In [19]:

df_cost = df_cost_realigned.copy()
df_vol = df_vol_realigned.copy()




def fixDuplicateProductNames(nameDict, frame):
    idList =[]
    for index, row in frame.iterrows():
        newId = 0
        if row['product_id'] not in nameDict.keys():
            newId = row['product_id']
            idList.append(newId)
            continue
        isSearching = True
        rowDecrementer= 1
        while isSearching: 
            #backtrack until you find a row that is not a duplicate product name 
            preceedingProduct = frame.at[index-rowDecrementer, "product"]
            preceedingClass = frame.at[index-rowDecrementer, "mail_class"]
            rowDecrementer +=1 
            if preceedingProduct not in allDuplicateProducts:
                isSearching = False
        #return correct product id based on the mail class of a preceeding row
        newId = list(nameDict.keys())[list(nameDict.values()).index(preceedingClass)]
        idList.append(newId)
    frame['product_id'] = idList
    frame = DataframeCleaning.realignWithKey(frame, keyFrame)
    return frame 



def fixDuplicateProductNames_agg(frame):
    for inputDict in allDuplicateDictionaries: 
        frame = fixDuplicateProductNames(inputDict, frame)

    return frame 

df_cost_fixDups = fixDuplicateProductNames_agg(df_cost)
df_vol_fixDups = fixDuplicateProductNames_agg(df_vol)




df_vol = df_vol_fixDups.copy()
df_cost= df_cost_fixDups.copy()

# df_vol[df_vol['product_id'].isin(allDuplicateDictionaries[2].keys())]
# df_cost[df_cost['product_id'].isin(allDuplicateDictionaries[1].keys())]

# Reassign Total Rows

In [20]:
df_vol = df_vol_fixDups.copy()
df_cost= df_cost_fixDups.copy()

summaryReassignMap = {
    80: 262,
    81: 263,
    82: 264,
    83: 265,
    91: 266,
}

# def reassignTotalRows 

def reassignTotalRows(frame):
    def applyReassign(row):
        if row in summaryReassignMap.keys():
            return summaryReassignMap[row]
        return row
    
    frame['product_id'] = frame['product_id'].apply(lambda row: applyReassign(row))

    frame = DataframeCleaning.realignWithKey(frame, keyFrame)
    return frame




df_vol_newTotals = reassignTotalRows(df_vol)
df_cost_newTotals = reassignTotalRows(df_cost)

# df_cost = df_cost_newTotals.copy()
# df_vol= df_vol_newTotals.copy()
# df_cost[df_cost['product_id'].isin(summaryReassignMap.values())]
# df_vol[df_vol['product_id'].isin(summaryReassignMap.values())]



In [21]:
df_cost = df_cost_newTotals.copy()
df_vol= df_vol_newTotals.copy()

def mergeVolCost(volFrame, costFrame):
    dropCols = ['product', 'mail_class']
    volFrame = volFrame.drop(columns = dropCols)
    costFrame = costFrame.drop(columns = dropCols)
    mergedFrame = pd.merge(volFrame, costFrame, on=["product_id", "fy"], how="outer")

    costCols = [c for c in costFrame.columns if c not in volFrame.columns]
    volCols = [c for c in volFrame.columns if c not in costFrame.columns]
    leadingCols = ['product_id', "fy"]

    finalColOrder = leadingCols + costCols + volCols
    mergedFrame = mergedFrame[finalColOrder]

    mergedFrame = DataframeCleaning.realignWithKey(mergedFrame, keyFrame)
    return mergedFrame

df_merged = mergeVolCost(df_vol, df_cost)
df_merged.head()

,product_id,product,mail_class,fy,revenue,attributable_cost,volume_variable_cost,product_specific_cost,rev_pc,attributable_cost_pc,cont_pc,cost_coverage,volume,weight,weight_pc
0,3,Single Piece Letters,First Class,2008,14353.524033,8443.075618748211,8434.32261874821,8.753,0.4283392480234627,0.2519590766159719,0.1763801714074908,1.7000349968591284,33509710.117,1000328.975,0.47763061942694274
1,4,Single Piece Cards,First Class,2008,500.489597,446.5087719559969,446.02677195599694,0.482,0.27114174086745557,0.2418974669332412,0.02924427393421436,1.12089532935161,1845859.643,11830.08,0.10254370136852274
2,5,Total Single Piece Letters and Cards,First Class,2008,14854.01363,8889.584390704207,8880.349390704207,9.235,0.42013220917755617,0.25143377552810814,0.16869843364944803,1.6709457919689432,35355569.76,1012159.0549999999,0.4580479112607009
3,8,Presort Letters,First Class,2008,16327.804358,5441.6680782620415,5429.030078262042,12.638,0.3374916702247292,0.11247793079057965,0.22501373943414954,3.000514570748088,48379873.634,2174874.013,0.7192657110113855
4,9,Presort Cards,First Class,2008,732.236796,282.27046183504393,281.3414618350439,0.929,0.20591601550888014,0.07937870524185411,0.12653731026702603,2.594096425250163,3555997.304,28969.254,0.13034544865335476


# All Functions -  Aggregate

In [22]:
df_vol_raw = FileIO.importFile(volExtractFileName)
df_cost_raw =FileIO.importFile(costExtractFileName)


transformFuncs = [
    removeSubstrings, 
    cleanProductNameCol, 
    labelFees, 
    matchProductName, 
    dropBlankRows, 
    dropUnmatchedProducts, 
    DataframeCleaning.realignWithKey, 
    fixDuplicateProductNames_agg 
]

def applyTransformations(frame, funcs):
    for func in funcs:
        frame = func(frame)
    return frame

df_vol_final = applyTransformations(df_vol_raw, transformFuncs)
df_cost_final = applyTransformations(df_cost_raw, transformFuncs)


df_merged = mergeVolCost(df_vol_final, df_cost_final)



# Load Finished File

In [23]:


FileIO.exportFile(df_merged, "transformFinished_separate.csv")

# Garbage

In [24]:
df = df_merged.copy()

# df[df['product'].str.contains("Spec")]
# df[df['product']]
# df.mail_class.value_counts()
# df[df['product'].str.contains("Total")]

countSer = df.groupby(["fy", "product_id"])['product_id'].count()
countSer.loc[countSer>1]


Series([], Name: product_id, dtype: int64)

In [25]:



# # groupSer  = df_cost.groupby(["fy", "product_id"])['product'].count()
# # groupSer  = df_vol.groupby(["fy", "product_id"])['product'].count()
# # groupSer.loc[groupSer>1]

# # df_vol['product'] = df_vol['product'].astype(str)
# # df_vol['product'] = df_vol['product'].astype(str)

# df_vol[df_vol['product'].str.contains("Comp")]['product'].value_counts()
# df_cost[df_cost['product'].str.contains("Comp")]['product'].value_counts()
# df_cost[df_cost['mail_class']=='Competitive']['product'].value_counts()
